# Riduzione della dimensionalità di dati Microarray
I dati molecolari sono estratti attraverso microarray. I file contenenti i dati molecolari iniziano tutti con la dicitura "data_" e contengono gli stessi dati normalizzati con procedure diverse. Questo perché i dati molecolari possono essere normalizzati in tanti modi e non si sa a priori quale tipologia di normalizzazione funziona meglio per la costruzione dell'autoencoder. Ognuno di questi file contiene una matrice in cui vi è una riga per ogni gene (feature di input) e una colonna per ogni paziente. Le prime due colonne contengono i nomi delle righe in formato diverso. La prima riga contiene i nomi delle colonne (tranne le prime due colonne i restanti nomi sono i codici dei campioni). I codici dei campioni sono sequenze di 15 caratteri.

## Estrazione dei dati dal dataset Microarray

Le strutture degli autoencoders sono disponibili nel modulo `autoencoder`. Le scelte architetturali ed implementative sono descritte all'interno della tesi. Prima di tutto, estraiamo i dati dal dataset Microarray attraverso il modulo `microarray`. Utilizzeremo di default il file `data_mRNA_median_Zscores.txt`, che ha fornito i risultati di compressione migliori. Il metodo `microarray.get_ds()`  restituisce il dataset già preprocessato (attraverso gli step descritti nella tesi).

In [ ]:
import pandas as pd 
import numpy  as np 

from modules import microarray  
from modules.autoencoder import *
from modules.preprocess  import *
from modules.utilities   import *
from modules.postprocess import *

ds = microarray.get_ds()
ds.head()

Estraiamo il numero di features, che servirà in futuro.

In [ ]:
nfeatures = ds.shape[1]

Salviamo il nome del file contente il dataset.

In [ ]:
dataset_file = microarray.get_data_sources()[0]

Salviamo il nome della cartella che conterrà i risultati.

In [ ]:
results_directory = 'results/microarray/blind'

## Estrazione degli autoencoders

Estraiamo gli autoencoders (blind):

In [ ]:
autoencoders_structures = blind_autoencoders(nfeatures)

## Preparazione dei dati di training

La tesi descrive accuratamente come avviene lo split e come avviene lo scaling (min-max normalization). 

In [ ]:
x_train, x_valid = train_test_split(ds, test_perc = .1)
x_train, x_test  = train_test_split(x_train, test_perc = .1)
x_train, x_test, scaler = normalize_sets(x_train, x_test)
x_valid = normalize_with_pretrained_scaler(scaler, x_valid)

## Embedding space di dimensione 150

Creiamo il modello `BLA150`: BLind Autoencoder 150 (embedding space dimension)

In [ ]:
emb150 = autoencoders_structures.get(150)
enc150 = emb150.get('encoder')
dec150 = emb150.get('decoder')
encpos = emb150.get('encoder_position')

BLA150 = generate_deep_autoencoder(nfeatures, enc150, dec150) 

In [ ]:
from keras.callbacks import EarlyStopping

earlystopping = EarlyStopping(monitor='val_loss', verbose=1, patience=20)

BLA150_training_hist = BLA150.fit(x_train, x_train,
            epochs=50, 
            batch_size=256,
            shuffle=True,
            verbose=0,
            callbacks=[earlystopping],
            validation_data=(x_test, x_test))

plot_history(BLA150_training_hist)

### Valutazione modello BLA150 

In [ ]:
training_loss = BLA150_training_hist.history['val_loss'][-1]
loss, mae, mse = BLA150.evaluate(x_valid, x_valid, verbose=0)
loss, mae, mse

In [ ]:
visualize_as_images(x_valid, BLA150, 10)

### Esportazione del dataset compresso da BLA150

In [ ]:
BLA150_encoder = extract_encoder(BLA150, encpos)
BLA150_ds = pd.DataFrame(BLA150_encoder.predict(np.asarray(ds).astype('float32')))
BLA150_ds.index = ds.index
BLA150_ds.head()

In [ ]:
save_compressed_dataset(BLA150_ds, loss, results_directory, dataset_file)

## Embedding space di dimensione 50

Creiamo il modello `BLA50`: BLind Autoencoder 50 (embedding space dimension)

In [ ]:
emb50 = autoencoders_structures.get(50)
enc50 = emb50.get('encoder')
dec50 = emb50.get('decoder')
encpos = emb50.get('encoder_position')

BLA50 = generate_deep_autoencoder(nfeatures, enc50, dec50) 

In [ ]:
from keras.callbacks import EarlyStopping

earlystopping = EarlyStopping(monitor='val_loss', verbose=1, patience=20)

BLA50_training_hist = BLA50.fit(x_train, x_train,
            epochs=50, 
            batch_size=256,
            shuffle=True,
            verbose=0,
            callbacks=[earlystopping],
            validation_data=(x_test, x_test))

plot_history(BLA50_training_hist)

### Valutazione modello BLA50 

In [ ]:
training_loss = BLA50_training_hist.history['val_loss'][-1]
loss, mae, mse = BLA50.evaluate(x_valid, x_valid, verbose=0)
loss, mae, mse

In [ ]:
visualize_as_images(x_valid, BLA50, 10)

### Esportazione del dataset compresso da BLA50

In [ ]:
BLA50_encoder = extract_encoder(BLA50, encpos)
BLA50_ds = pd.DataFrame(BLA50_encoder.predict(np.asarray(ds).astype('float32')))
BLA50_ds.index = ds.index
BLA50_ds.head()

In [ ]:
save_compressed_dataset(BLA50_ds, loss, results_directory, dataset_file)

## Embedding space di dimensione 25

Creiamo il modello `BLA25`: BLind Autoencoder 25 (embedding space dimension)

In [ ]:
emb25 = autoencoders_structures.get(25)
enc25 = emb25.get('encoder')
dec25 = emb25.get('decoder')
encpos = emb25.get('encoder_position')

BLA25 = generate_deep_autoencoder(nfeatures, enc25, dec25) 

In [ ]:
from keras.callbacks import EarlyStopping

earlystopping = EarlyStopping(monitor='val_loss', verbose=1, patience=20)

BLA25_training_hist = BLA25.fit(x_train, x_train,
            epochs=50, 
            batch_size=256,
            shuffle=True,
            verbose=1,
            callbacks=[earlystopping],
            validation_data=(x_test, x_test))

plot_history(BLA25_training_hist)

### Valutazione modello BLA25

In [ ]:
training_loss = BLA25_training_hist.history['val_loss'][-1]
loss, mae, mse = BLA25.evaluate(x_valid, x_valid, verbose=0)
loss, mae, mse

In [ ]:
visualize_as_images(x_valid, BLA25, 10)

### Esportazione del dataset compresso da BLA25

In [ ]:
BLA25_encoder = extract_encoder(BLA25, encpos)
BLA25_ds = pd.DataFrame(BLA25_encoder.predict(np.asarray(ds).astype('float32')))
BLA25_ds.index = ds.index
BLA25_ds.head()

In [ ]:
save_compressed_dataset(BLA25_ds, loss, results_directory, dataset_file)